In [ ]:
import numpy as np
from imutils import contours
import cv2
import os

root_directory = r"C:\Users\User\Downloads\Votes.v4i.folder\train"
for subdir, dirs, files in os.walk(root_directory):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(subdir, file)
            image = cv2.imread(image_path)
            dim=(1398,1783)
            image=cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


            cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                area = cv2.contourArea(c)
                if area <50:
                    cv2.drawContours(thresh, [c], -1, 0, -1)


            invert = 255 - thresh
            offset, old_cY, old_cX, first = 10, 0, 0, True
            visualize = cv2.cvtColor(invert, cv2.COLOR_GRAY2BGR)
            
            cnts = cv2.findContours(invert, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            (cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")
            crosscount=-1
            table = []
            row = []
            prevX=0
            current_row_index = -1 
            current_column_index = -1
            column_centroids = []
            value= 0
            value1=0
            approxval=None
            processed_centroids = []  
            outx=0
            outy=0
            size=0
            party= None
            terminate_main_loop= False
            for c in cnts:
                M = cv2.moments(c)
                if M["m00"] > 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])

                    if 0 <= cX <= 1000 or 0<=cY<=100:
                        continue

                    too_close = False
                    for (prev_cX, prev_cY) in processed_centroids:
                        if abs(cX - prev_cX) < 200 and abs(cY - prev_cY) < 200: 
                            too_close = True
                            break

                    if too_close:
                        continue  


                    processed_centroids.append((cX, cY))

                    if abs(cY - old_cY) > offset:
                        if not first:
                            table.append(row)
                        old_cY = cY
                        row = []
                        current_row_index += 1


                    column_index = -1
                    for i, column_cX in enumerate(column_centroids):
                        if abs(cX - column_cX) < 50:
                            column_index = i
                            break

                    if column_index == -1:
                        column_index = len(column_centroids)
                        column_centroids.append(cX)

                    if column_index ==0:
                        
                        x, y, w, h= cv2.boundingRect(c)
                        
                        roi =invert[y:y + h, x:x + w]
                        crosscount=crosscount+1
                        roi_cnts = cv2.findContours(roi, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                        roi_cnts = roi_cnts[0] if len(roi_cnts) == 2 else roi_cnts[1]

                        for roi_c in roi_cnts:
                                
                                epsilon = 0.04 * cv2.arcLength(roi_c, True)
                                approx = cv2.approxPolyDP(roi_c, epsilon, True)
                                num_vertices = len(approx)
                                if(num_vertices>4):
                                    
                                    if value<2:
                                        print(f'num vertices {num_vertices} cx{cX} prevx {prevX}')
                                        if(num_vertices==8):
                                            
                                            if prevX==0:
                                                
                                                value=value+1
                                                prevX=cX
                                                value1=crosscount
                                                approxval=approx
                                                size=len(approx)
                                                outx=x
                                                outy=y
                                            else:
                                                if prevX==cX:
                                                    value=value-1
                                                    value1=crosscount
                                                    approxval=approx
                                                    size=len(approx)
                                                    outx=x
                                                    outy=y
                                                    value=value+1
                                                else:
                                                    value=value+1

                                        else:
                                            if prevX!=cX:
                                                value=value+1
                                                prevX=cX
                                                
                                            
                                            
                                    else:
                                        
                                        terminate_main_loop=True
                                        break
                                         
                    if terminate_main_loop:
                        break
            
            if value==1:
                x2,y2, w2, h2 = cv2.boundingRect(approxval)
                x2 += outx
                y2 += outy
                cv2.rectangle(visualize, (x2, y2), (x2 +w2, y2 + h2), (0, 0, 255)), 
                cv2.putText(
                                                visualize,
                                                f"GG: ({value1}, {0})",
                                                (x2-20,y2-20),
                                                cv2.FONT_HERSHEY_SIMPLEX,
                                                0.8,
                                                (0,0,255),
                                                2,
                                            )
                
                
                print(f'X value :{outx} Number of crosses:{value} Row index {value1} size{size}')
                if size==8:
                    
                    if value1==0:
                        party="PartyA"
                    elif value1==1:
                        party="PartyB"
                    elif value1==2:
                        party="PartyC"
                    elif value1==3:
                        party="PartyD"
                    elif value1==4:
                        party="PartyE"
                    else:
                        party="PartyF"
                    
            print(party)
            scale_percent = 40 
            width = int(visualize.shape[1] * scale_percent / 100)
            height = int(visualize.shape[0] * scale_percent / 100)
            dim = (width, height)




            resized_visualize = cv2.resize(visualize, dim, interpolation=cv2.INTER_AREA)
            height, width, channels =resized_visualize.shape
            cv2.imshow('Visualize', resized_visualize)
            cv2.waitKey(0)
            cv2.destroyAllWindows()


In [8]:
import numpy as np
from imutils import contours
import cv2


image = cv2.imread(r"C:\Users\User\Desktop\New Votes DataSet\New Votes DataSet\Invalid Votes\1733843944290.jpg")
dim=(1398,1783)
image=cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area <10:
        cv2.drawContours(thresh, [c], -1, 0, -1)


invert = 255 - thresh
offset, old_cY, old_cX, first = 10, 0, 0, True
visualize = cv2.cvtColor(invert, cv2.COLOR_GRAY2BGR)


cnts = cv2.findContours(invert, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
(cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")
crosscount = -1
table = []
row = []
prevX = 0
current_row_index = -1
current_column_index = -1
column_centroids = []
value = 0
value1 = 0
approxval = None
processed_centroids = []
outx = 0
outy = 0
size = 0
party = None
terminate_main_loop = False

for c in cnts:

    M = cv2.moments(c)
    if M["m00"] > 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        if 0 <= cX <= 1000 or 0 <= cY <= 100:
            continue

        too_close = False
        for (prev_cX, prev_cY) in processed_centroids:
            if abs(cX - prev_cX) < 200 and abs(cY - prev_cY) < 200: 
                too_close = True
                break

        if too_close:
            continue  

        processed_centroids.append((cX, cY))

        if abs(cY - old_cY) > offset:
            if not first:
                table.append(row)
            old_cY = cY
            row = []
            current_row_index += 1

        column_index = -1
        for i, column_cX in enumerate(column_centroids):
            if abs(cX - column_cX) < 50:
                column_index = i
                break

        if column_index == -1:
            column_index = len(column_centroids)
            column_centroids.append(cX)

        if column_index == 0:

            x, y, w, h = cv2.boundingRect(c)

            roi = invert[y:y + h, x:x + w]
            crosscount += 1
            roi_cnts = cv2.findContours(roi, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            roi_cnts = roi_cnts[0] if len(roi_cnts) == 2 else roi_cnts[1]

            for roi_c in roi_cnts:

                epsilon = 0.04 * cv2.arcLength(roi_c, True)
                approx = cv2.approxPolyDP(roi_c, epsilon, True)
                num_vertices = len(approx)
                if num_vertices > 4:

                    if value < 2:
                        print(f'num vertices {num_vertices} cx{cX} prevx {prevX}')
                        if num_vertices == 8:

                            if prevX == 0:
                                value += 1
                                prevX = cX
                                value1 = crosscount
                                approxval = approx
                                size = len(approx)
                                outx = x
                                outy = y
                            else:
                                if prevX == cX:
                                    value -= 1
                                    value1 = crosscount
                                    approxval = approx
                                    size = len(approx)
                                    outx = x
                                    outy = y
                                    value += 1
                                else:
                                    value += 1

                        else:
                            if prevX != cX:
                                value += 1
                                prevX = cX

                    else:

                        terminate_main_loop = True
                        break

            if terminate_main_loop:
                break

if value == 1:
    x2, y2, w2, h2 = cv2.boundingRect(approxval)
    x2 += outx
    y2 += outy
    cv2.rectangle(visualize, (x2, y2), (x2 + w2, y2 + h2), (0, 0, 255))
    cv2.putText(
        visualize,
        f"GG: ({value1}, {0})",
        (x2 - 20, y2 - 20),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.8,
        (0, 0, 255),
        2,
    )

    print(f'X value :{outx} Number of crosses:{value} Row index {value1} size{size}')
    if size == 8:
        if value1 == 0:
            party = "PartyA"
        elif value1 == 1:
            party = "PartyB"
        elif value1 == 2:
            party = "PartyC"
        elif value1 == 3:
            party = "PartyD"
        elif value1 == 4:
            party = "PartyE"
        else:
            party = "PartyF"


scale_percent = 40 
width = int(visualize.shape[1] * scale_percent / 100)
height = int(visualize.shape[0] * scale_percent / 100)
dim = (width, height)


resized_visualize = cv2.resize(visualize, dim, interpolation=cv2.INTER_AREA)


cv2.imshow('Visualize', resized_visualize)
cv2.waitKey(0)
cv2.destroyAllWindows() 

num vertices 8 cx1213 prevx 0
num vertices 7 cx1213 prevx 1213
num vertices 6 cx1213 prevx 1213
num vertices 6 cx1211 prevx 1213


In [1]:
import os
import random
import shutil

original_dataset_path = r"C:\Users\User\Desktop\New Votes DataSet\New Votes DataSet"

output_base_path = r"C:\Users\User\Desktop\nm"
os.makedirs(output_base_path, exist_ok=True)

num_datasets = 17

for i in range(1, num_datasets + 1):
    dataset_path = os.path.join(output_base_path, f'Dataset_{i}') #Dataset_1,
    os.makedirs(dataset_path, exist_ok=True)

    for cls in os.listdir(original_dataset_path):
        class_path = os.path.join(original_dataset_path, cls)
        if os.path.isdir(class_path):
            
            images = os.listdir(class_path)
            num_images = random.randint(1, len(images))
            selected_images = random.sample(images, num_images)

            
            new_class_path = os.path.join(dataset_path, cls)
            os.makedirs(new_class_path, exist_ok=True)

            
            for image in selected_images:
                src = os.path.join(class_path, image)
                dst = os.path.join(new_class_path, image)
                shutil.copy(src, dst)



Generated 17 datasets at C:\Users\User\Desktop\nm


In [2]:
import os
import numpy as np
from imutils import contours
import cv2
from mesa import Agent, Model
from mesa.time import BaseScheduler
import random

class DatasetAgent(Agent):
    def __init__(self, unique_id, model, dataset_id, dataset_path):
        super().__init__(unique_id, model)
        self.dataset_id = dataset_id
        self.dataset_path = dataset_path
        self.local_vote_counts = {
            "PartyA": 0,
            "PartyB": 0,
            "PartyC": 0,
            "PartyD": 0,
            "PartyE": 0,
            "PartyF": 0
        }

    def process_images(self):
        # Gather all images in the dataset and shuffle
        all_images = []
        for subdir, dirs, files in os.walk(self.dataset_path):
            for file in files:
                if file.endswith(('.jpg', '.png', '.jpeg')):
                    all_images.append(os.path.join(subdir, file))
        
        random.shuffle(all_images)  # Shuffle images

        # Process each shuffled image
        for image_path in all_images:
            image = cv2.imread(image_path)
            dim = (1398, 1783)
            image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

            cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                area = cv2.contourArea(c)
                if area < 50:
                    cv2.drawContours(thresh, [c], -1, 0, -1)

            invert = 255 - thresh
            offset, old_cY, old_cX, first = 10, 0, 0, True
            cnts = cv2.findContours(invert, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            (cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")

            crosscount = -1
            table = []
            row = []
            prevX = 0
            current_row_index = -1
            current_column_index = -1
            column_centroids = []
            value = 0
            value1 = 0
            approxval = None
            processed_centroids = []
            outx = 0
            outy = 0
            size = 0
            party = None
            terminate_main_loop = False

            for c in cnts:
                M = cv2.moments(c)
                if M["m00"] > 0:
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])

                    if 0 <= cX <= 1000 or 0 <= cY <= 100:
                        continue

                    too_close = False
                    for (prev_cX, prev_cY) in processed_centroids:
                        if abs(cX - prev_cX) < 200 and abs(cY - prev_cY) < 200:
                            too_close = True
                            break

                    if too_close:
                        continue

                    processed_centroids.append((cX, cY))

                    if abs(cY - old_cY) > offset:
                        if not first:
                            table.append(row)
                        old_cY = cY
                        row = []
                        current_row_index += 1

                    column_index = -1
                    for i, column_cX in enumerate(column_centroids):
                        if abs(cX - column_cX) < 50:
                            column_index = i
                            break

                    if column_index == -1:
                        column_index = len(column_centroids)
                        column_centroids.append(cX)

                    if column_index == 0:
                        x, y, w, h = cv2.boundingRect(c)
                        roi = invert[y:y + h, x:x + w]
                        crosscount += 1
                        roi_cnts = cv2.findContours(roi, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                        roi_cnts = roi_cnts[0] if len(roi_cnts) == 2 else roi_cnts[1]

                        for roi_c in roi_cnts:
                            epsilon = 0.04 * cv2.arcLength(roi_c, True)
                            approx = cv2.approxPolyDP(roi_c, epsilon, True)
                            num_vertices = len(approx)
                            if num_vertices > 4:
                                if value < 2:
                                    if num_vertices == 8:
                                        if prevX == 0:
                                            value += 1
                                            prevX = cX
                                            value1 = crosscount
                                            approxval = approx
                                            size = len(approx)
                                            outx = x
                                            outy = y
                                        else:
                                            if prevX == cX:
                                                value -= 1
                                                value1 = crosscount
                                                approxval = approx
                                                size = len(approx)
                                                outx = x
                                                outy = y
                                                value += 1
                                            else:
                                                value += 1
                                    else:
                                        if prevX != cX:
                                            value += 1
                                            prevX = cX
                                else:
                                    terminate_main_loop = True
                                    break

                    if terminate_main_loop:
                        break

            if value == 1:
                if size == 8:
                    if value1 == 0:
                        party = "PartyA"
                    elif value1 == 1:
                        party = "PartyB"
                    elif value1 == 2:
                        party = "PartyC"
                    elif value1 == 3:
                        party = "PartyD"
                    elif value1 == 4:
                        party = "PartyE"
                    else:
                        party = "PartyF"

                if party:
                    self.local_vote_counts[party] += 1

    def step(self):
        self.process_images()
        print(f"Dataset_{self.dataset_id} vote counts: {self.local_vote_counts}")

class MainModel(Model):
    def __init__(self, num_agents, dataset_root):
        self.num_agents = num_agents
        self.schedule = BaseScheduler(self)
        self.dataset_root = dataset_root

        # Create agents, each handling its respective dataset
        for i in range(1, num_agents + 1):
            dataset_path = os.path.join(self.dataset_root, f"Dataset_{i}")
            agent = DatasetAgent(i, self, i, dataset_path)
            self.schedule.add(agent)

    def step(self):
        for agent in self.schedule.agents:
            agent.step()

if __name__ == "__main__":
    dataset_root = r"C:\Users\User\Desktop\New Votes DataSet"
    num_agents = 17  # Number of datasets/agents
    model = MainModel(num_agents, dataset_root)

    model.step()


C:\Users\User\AppData\Local\Temp\ipykernel_12416\2302759800.py:11: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


Dataset_1 vote counts: {'PartyA': 39, 'PartyB': 100, 'PartyC': 80, 'PartyD': 97, 'PartyE': 46, 'PartyF': 91}
Dataset_2 vote counts: {'PartyA': 38, 'PartyB': 103, 'PartyC': 90, 'PartyD': 85, 'PartyE': 30, 'PartyF': 80}
Dataset_3 vote counts: {'PartyA': 65, 'PartyB': 61, 'PartyC': 75, 'PartyD': 34, 'PartyE': 66, 'PartyF': 97}
Dataset_4 vote counts: {'PartyA': 38, 'PartyB': 63, 'PartyC': 95, 'PartyD': 73, 'PartyE': 93, 'PartyF': 11}


KeyboardInterrupt: 

In [ ]:
pip install imutils

In [ ]:
pip install pytesseract